In [1]:
import seaborn as sns
from scipy.stats import pearsonr, ttest_ind
import pandas as pd
from selenium import webdriver  
from selenium.webdriver.chrome.service import Service  
from selenium.webdriver.common.by import By  
from selenium.webdriver.chrome.options import Options  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import time  
import random  
import matplotlib.pyplot as plt
from selenium.common.exceptions import ElementClickInterceptedException
import matplotlib.ticker as mtick
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from scipy.stats import f_oneway
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

/Users/johnreisfield/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Load the salary cap data from the CSV file
salary_cap_df = pd.read_excel(r"NHL Salary Cap.xlsx")
salary_cap_df['Season'] = salary_cap_df['Season'].astype(str)
salary_cap_df['Injured'] = salary_cap_df['Injured'].astype(float)
salary_cap_df['Long0term IR'] = salary_cap_df['Long0term IR'].astype(float)
# Display the first few rows to verify
print(salary_cap_df.head())

# Convert the 'Season' column to integers and drop rows with NaN values
salary_cap_df = salary_cap_df.dropna(subset=['Season'])
salary_cap_df['Season'] = salary_cap_df['Season'].astype(str).str.replace('.0', '', regex=False)

# Mapping of abbreviated team names in salary_cap_df to full team names
team_name_mapping = {
    'ANA': 'Anaheim Ducks',
    'CHI': 'Chicago Blackhawks',
    'ARI': 'Arizona Coyotes',
    'BUF': 'Buffalo Sabres',
    'SJS': 'San Jose Sharks',
    'CBJ': 'Columbus Blue Jackets',
    'CGY': 'Calgary Flames',
    'DET': 'Detroit Red Wings',
    'WAS': 'Washington Capitals',
    'SEA': 'Seattle Kraken',
    'NJD': 'New Jersey Devils',
    'NSH': 'Nashville Predators',
    'STL': 'St. Louis Blues',
    'MIN': 'Minnesota Wild',
    'NYR': 'New York Rangers',
    'MTL': 'Montréal Canadiens',
    'VGK': 'Vegas Golden Knights',
    'NYI': 'New York Islanders',
    'VAN': 'Vancouver Canucks',
    'TOR': 'Toronto Maple Leafs',
    'COL': 'Colorado Avalanche',
    'OTT': 'Ottawa Senators',
    'PIT': 'Pittsburgh Penguins',
    'TBL': 'Tampa Bay Lightning',
    'BOS': 'Boston Bruins',
    'LAK': 'Los Angeles Kings',
    'DAL': 'Dallas Stars',
    'EDM': 'Edmonton Oilers',
    'CAR': 'Carolina Hurricanes',
    'WPG': 'Winnipeg Jets',
    'FLA': 'Florida Panthers',
    'PHI': 'Philadelphia Flyers',
    'PHX': 'Arizona Coyotes',  # Phoenix Coyotes is now Arizona Coyotes
}
salary_cap_df['Team'] = salary_cap_df['Team'].replace(team_name_mapping)

# Strip from the Team column in salary_cap_df
salary_cap_df['Team'] = salary_cap_df['Team'].str.strip()

# Map the abbreviated team names in salary_cap_df to the full team names in stats
salary_cap_df['Team'] = salary_cap_df['Team'].replace(team_name_mapping)



  Season  Team  Total Players  Average Age  Cap Space  Total Cap  \
0   2023   ANA             55         25.3   13168855   70331145   
1   2023   CHI             54         27.3   12857222   70642778   
2   2023   ARI             54         25.9   12180382   71319618   
3   2023   BUF             51         25.0   10676071   72823929   
4   2023   SJS             62         27.1    9725679   73749321   

   Long0term IR    Active     Injured  
0           0.0  62409004   2750000.0  
1           0.0  51951617   6000000.0  
2           0.0  42536666  23148206.0  
3           0.0  66673872   4285714.0  
4           0.0  48219231  13425000.0  
